In [1]:
import open_clip

e:\ELFAK\MASTER RAD\MASTER_RAD-Vector_Search\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

In [3]:
encoder_model, _, preprocess = open_clip.create_model_and_transforms('xlm-roberta-base-ViT-B-32', pretrained='laion5B-s13B-b90k')
tokenizer = open_clip.get_tokenizer('xlm-roberta-base-ViT-B-32')

In [3]:
encoder_model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained='laion2b_s32b_b82k')
tokenizer = open_clip.get_tokenizer('ViT-L-14')


In [7]:
encoder_model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14', pretrained='laion2b_s32b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-H-14')


In [2]:
#ViT-g-14', 'laion2b_s34b_b88k

encoder_model, _, preprocess = open_clip.create_model_and_transforms('ViT-g-14', pretrained='laion2b_s34b_b88k')
tokenizer = open_clip.get_tokenizer('ViT-g-14')


In [3]:
encoder_model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-39): 40 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1408, out_features=1408, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1408,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1408, out_features=6144, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1408,), eps=1e-05, elementwi

In [4]:
encoder_model.token_embedding

Embedding(49408, 1024)

In [28]:
def prep_custom_model(model):
    """ Freeze layers of the CLIP model (inplace) for finetuning"""

    # freeze all layers of the network
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze layers in image embedding network
    for param in model.visual.transformer.resblocks[11].parameters():
        param.requires_grad = True

    for param in model.visual.ln_post.parameters():
        param.requires_grad = True

    # Unfreeze layers in text embedding network
    for param in model.text.transformer.encoder.layer[11].parameters():
        param.requires_grad = True

    for param in model.text.pooler.parameters():
        param.requires_grad = True

    for param in model.text.proj.parameters():
        param.requires_grad = True


In [25]:
encoder_model.visual.transformer.resblocks[11].parameters

<bound method Module.parameters of ResidualAttentionBlock(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (ls_1): Identity()
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): Sequential(
    (c_fc): Linear(in_features=768, out_features=3072, bias=True)
    (gelu): GELU(approximate='none')
    (c_proj): Linear(in_features=3072, out_features=768, bias=True)
  )
  (ls_2): Identity()
)>

In [29]:
prepared_model = prep_custom_model(encoder_model)

## XLM ROBERTA LARGE VIT H 14

In [4]:
encoder_model, _, preprocess = open_clip.create_model_and_transforms('xlm-roberta-large-ViT-H-14', pretrained='frozen_laion5b_s13b_b90k')
tokenizer = open_clip.get_tokenizer('xlm-roberta-large-ViT-H-14')

In [3]:
encoder_model, _, preprocess = open_clip.create_model_and_transforms('EVA02-E-14-plus', pretrained='laion2b_s9b_b144k')
tokenizer = open_clip.get_tokenizer('EVA02-E-14-plus')

In [ ]:
import open_clip

model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms('hf-hub:laion/CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k')
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-H-14-frozen-xlm-roberta-large-laion5B-s13B-b90k')

In [3]:
import torch
from PIL import Image
import open_clip

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

#image = preprocess(Image.open("CLIP.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    #image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    print(text_features)
    #image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    print(text_features)

    #text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

#print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

tensor([[-0.0319, -0.3413, -0.4652,  ...,  0.1816,  0.0024,  0.1847],
        [-0.2863, -0.0884, -0.1613,  ...,  0.4537, -0.1437,  0.2951],
        [-0.0330, -0.0352,  0.0300,  ...,  0.1025,  0.0840,  0.2781]])
tensor([[-0.0033, -0.0355, -0.0484,  ...,  0.0189,  0.0002,  0.0192],
        [-0.0298, -0.0092, -0.0168,  ...,  0.0473, -0.0150,  0.0307],
        [-0.0036, -0.0038,  0.0033,  ...,  0.0112,  0.0091,  0.0302]])


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-large")

modules.json: 100%|██████████| 387/387 [00:00<00:00, 236kB/s]
README.md: 100%|██████████| 160k/160k [00:00<00:00, 759kB/s]
sentence_bert_config.json: 100%|██████████| 57.0/57.0 [00:00<00:00, 219kB/s]
model.safetensors: 100%|██████████| 2.24G/2.24G [03:49<00:00, 9.77MB/s]
tokenizer_config.json: 100%|██████████| 418/418 [00:00<00:00, 1.91MB/s]
sentencepiece.bpe.model: 100%|██████████| 5.07M/5.07M [00:00<00:00, 10.6MB/s]
tokenizer.json: 100%|██████████| 17.1M/17.1M [00:01<00:00, 11.2MB/s]
1_Pooling/config.json: 100%|██████████| 201/201 [00:00<00:00, 918kB/s]
